In [309]:
import pandas as pd
import numpy as np
from nltk import word_tokenize
import nltk
from string import punctuation
from sklearn.decomposition import NMF
import json
from sklearn.metrics.pairwise import cosine_similarity


# Vetorização
from sklearn.feature_extraction.text import TfidfVectorizer, TfidfTransformer

In [2]:
stopwords = nltk.corpus.stopwords.words('portuguese')
numbers = '0123456789'

In [349]:
df = pd.read_csv('noticias_globo.csv')
df.tail()

,link,img_url,titulo,subtitulo,descricao,doc
17816,https://g1.globo.com/politica/noticia/senado-a...,https://s.glbimg.com/jo/g1/static/live/imagens...,Senado aprova incluir 12 cidades na região int...,"Criada em 1998, região tem o objetivo de artic...",Foram 64 votos a favor da proposta e um único...,senado aprova incluir cidades região integrada...
17817,https://globoesporte.globo.com/futebol/times/c...,https://s2.glbimg.com/Nww_eNqyeLPYpZi1NaIX4nDu...,Carille troca o Corinthians por time da Arábia...,"Anunciado como substituto, Osmar Loss comanda ...",– Ele aceitou a proposta – escreveu Andrés. ...,carille troca corinthians time arábia saudita ...
17818,https://sportv.globo.com/site/programas/copa-2...,https://s2.glbimg.com/DZrhAjOV0BGcouOHP7dDLaKP...,Goleiro titular da Argentina sofre lesão e est...,Romero teve papel decisivo no time vice-campeã...,"Segundo a imprensa argentina, Romero deve pas...",goleiro titular argentina sofre lesão copa rom...
17819,https://g1.globo.com/economia/noticia/operacao...,https://s2.glbimg.com/wzA7nThsl0H7H7D1miRUz1TP...,Anatel apreende mais de 10 mil produtos em açã...,15 empresas foram alvo da fiscalização realiza...,A operação ocorreu em 14 cidades de 7 estados...,anatel apreende 10 mil produtos ação contra pi...
17820,https://g1.globo.com/economia/noticia/desonera...,https://s.glbimg.com/jo/g1/static/live/imagens...,Governo anuncia acordo para zerar um dos tribu...,NaN,"No mesmo anúncio, Guardia também informou que...",governo anuncia acordo zerar tributos sobre di...


In [278]:
def processa(row):
    txt = row['titulo'] + ' ' + ("" if pd.isnull(row['subtitulo']) else row['subtitulo']) + ' ' + ("" if pd.isnull(row['descricao']) else row['descricao'])
    
    return ' '.join([t for t in word_tokenize(txt.lower()) if (t not in stopwords) and (t not in punctuation + numbers)])

df['doc'] = df.apply(processa, axis=1)

df['doc'].head()

0    médicos divulga lista cidades receberam substi...
1    temer assina reajuste 16,38 ministros stf pres...
2    onu libera r 35,5 milhões ajuda humanitária ve...
3    sonda nasa pousa marte veja 1ª imagem planeta ...
4    cristiana lôbo bolsonaro pretende centralizar ...
Name: doc, dtype: object

In [344]:
def trazer_mais_proximas(noticia, quantidade=5):    
    vectorizer = TfidfVectorizer(
        analyzer='word',
        ngram_range=(1, 1),
        max_features=3500,
        binary=False,
        use_idf=True
    )
    data = df.append({'doc': noticia}, ignore_index=True)
    tfidf_matrix = vectorizer.fit_transform(data['doc'])

    # Calcula todas as similaridades de cada linha
    sim = cosine_similarity(tfidf_matrix,)
    
    dfs = pd.DataFrame(sim[:][-1])
    
    dfs = dfs.sort_values(by=[0], ascending=False)
    
    l = dfs.index.values.tolist()[1:quantidade]

    return pd.DataFrame([data.iloc[x] for x in l])

In [345]:
info = trazer_mais_proximas("bolsonaro", 100)
info

,link,img_url,titulo,subtitulo,descricao,doc
1884,https://g1.globo.com/politica/noticia/2018/11/...,https://s2.glbimg.com/-cquNmPOiwL-Czrbd7wbQYoO...,Bolsonaro acompanha culto em Igreja Batista no...,Presidente eleito foi a culto acompanhado da m...,"Ao final do evento, Bolsonaro subiu no palco ...",bolsonaro acompanha culto igreja batista rio p...
2402,https://g1.globo.com/rj/rio-de-janeiro/eleicoe...,https://s2.glbimg.com/kh7cspCJlSYYJmGd8ZyBkOiq...,"No 1º dia como eleito, Bolsonaro fica em casa ...",Policiamento na entrada de condomínio foi refo...,"Candidato pelo PSL, o capitão reformado do Ex...",1º dia eleito bolsonaro fica casa recebe aliad...
3382,https://g1.globo.com/fato-ou-fake/noticia/2018...,https://s2.glbimg.com/rO4XoaNDEsRhJGgCgqr2UtrP...,É #FAKE que Bolsonaro simulou facada para disf...,NaN,Fake — Foto: G1 O médico Antonio Luiz de Vas...,é fake bolsonaro simulou facada disfarçar cânc...
2884,https://g1.globo.com/politica/eleicoes/2018/no...,https://s2.glbimg.com/WnTxnHgth4H5IoPYeE_m-kT8...,Bolsonaro recebe deputados e propõe prontuário...,NaN,Jair Bolsonaro não teve atividades de campanh...,bolsonaro recebe deputados propõe prontuário e...
4298,https://g1.globo.com/rj/rio-de-janeiro/eleicoe...,https://s2.glbimg.com/wl6wtiW3tAqdscnongf_RKR-...,"Após votar no Rio, Bolsonaro diz acreditar que...",NaN,Bolsonaro chegou às 8h55 à zona eleitoral. El...,após votar rio bolsonaro diz acreditar vencerá...
3641,https://g1.globo.com/politica/eleicoes/2018/no...,https://s.glbimg.com/jo/g1/static/live/imagens...,Bolsonaro visita batalhão do Bope no RJ e se r...,NaN,Ele conversou com integrantes da coorporação ...,bolsonaro visita batalhão bope rj reúne polici...
2390,https://g1.globo.com/politica/eleicoes/2018/no...,https://s2.glbimg.com/esiOUs8FGIbgb4sNc1JNQOOv...,Saiba quem é quem no núcleo político de Bolsonaro,NaN,"Com a assessoria do grupo, o candidato do PSL...",saiba é núcleo político bolsonaro assessoria g...
2430,https://g1.globo.com/politica/eleicoes/2018/no...,https://s2.glbimg.com/FWGFBN9VolEcHnKwT4y5Sqsy...,FOTOS: eleitores de Bolsonaro celebram com buz...,NaN,Apoiadores se concentram na frente da casa de...,fotos eleitores bolsonaro celebram buzinaços f...
4747,https://g1.globo.com/rj/rio-de-janeiro/eleicoe...,https://s2.glbimg.com/IWoM2WzC5wOsGMbNW0zHsz7e...,'Bolsonaro não fez 10% do que eu fiz para o RJ...,NaN,"""O Bolsonaro está com uma boa votação no Rio ...",'bolsonaro fez 10 fiz rj diz haddad `` bolsona...
6837,https://g1.globo.com/politica/eleicoes/2018/no...,https://s2.glbimg.com/Y8HGuRJKt7ckbI0kcEcUKJC5...,VÍDEO: Bolsonaro relata 'dor insuportável' apó...,"'Nunca fiz mal a ninguém', afirmou ele em grav...","""Até o momento, Deus quis assim. Eu me prepara...",vídeo bolsonaro relata 'dor insuportável após ...


In [13]:
def vetorizer(doc):
    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform(doc)

    return (tfidf_matrix, tfidf_vectorizer)

In [11]:
#processa(df)
vetorizer(df['doc'])[0]

<1000x8221 sparse matrix of type '<class 'numpy.float64'>'
	with 30105 stored elements in Compressed Sparse Row format>

In [9]:
def topic_modeling(doc, num_topics=5):
    tfidf_matrix, tfidf_vectorizer = vetorizer(doc)
    
    tfidf_feature_names = tfidf_vectorizer.get_feature_names()

    nmf_model = NMF(n_components=num_topics).fit(tfidf_matrix)

    # Matriz de tópicos x documentos (W)
    nmf_W = nmf_model.transform(tfidf_matrix)

    # Matriz de palavras x tópicos (H)
    nmf_H = nmf_model.components_

    return (nmf_H, nmf_W, tfidf_feature_names)

In [10]:
# Função para obter os tópicos
def obtem_topicos(H, W, feature_names, documents, num_top_words=5, num_top_documents=3):
    topics = {}
    topics['quantidade'] = len(H)
    topics['topics'] = []
    for topic_idx, topic in enumerate(H):
        doc = {}
        doc['terms'] = [(feature_names[i], round(H[topic_idx][i], 2)) for i in topic.argsort()[:-num_top_words - 1:-1]]     

        # Top documentos relacionados   
        doc['top_docs'] = []
        
        top_doc_indices = np.argsort( W[:,topic_idx] )[::-1][0:num_top_documents]
        for doc_index in top_doc_indices:

            # Id do documento
            id_doc = int(doc_index)            
            
            top_doc = pd.Series.to_json(documents.iloc[doc_index], force_ascii=False)
            
            doc['top_docs'].append(json.loads(top_doc))
            
        topics['topics'].append(doc)
    return topics

nmf_H, nmf_W, tfidf_feature_names = topic_modeling(info['doc'])       
obtem_topicos(nmf_H, nmf_W, tfidf_feature_names, info)

/root/miniconda3/lib/python3.7/site-packages/sklearn/decomposition/nmf.py:116: RuntimeWarning: invalid value encountered in sqrt
  return np.sqrt(res * 2)


{'quantidade': 5,
 'topics': [{'terms': [('general', 1.03),
    ('santos', 0.86),
    ('cruz', 0.86),
    ('civil', 0.69),
    ('secretaria', 0.69)],
   'top_docs': [{'link': 'https://g1.globo.com/politica/noticia/2018/11/26/bolsonaro-anuncia-o-general-carlos-alberto-dos-santos-cruz-como-ministro-da-secretaria-de-governo.ghtml',
     'img_url': 'https://s2.glbimg.com/BlgiQBl0Bs75o-rPsiryDzpvn8g=/0x0:1024x683/984x0/smart/filters:strip_icc()/i.s3.glbimg.com/v1/AUTH_59edd422c0c84a879bd37670ae4f538a/internal_photos/bs/2018/a/T/7wC4u1T4m8xwq27MPuVg/santos-cruz.jpg',
     'titulo': 'Bolsonaro anuncia general como ministro da Secretaria de Governo',
     'subtitulo': None,
     'descricao': 'None Bolsonaro anuncia o general Carlos Alberto dos Santos Cruz futuro secretário de governo Os ministros de BolsonaroNoneNoneNone O ministro que coordena a transição e futuro chefe da Casa Civil, Onyx Lorenzoni, afirmou há duas semanas que a Secretaria de Governo seria incorporada pela Casa Civil. Assim,